# Bag of Words Meets Bags of Popcorn

In this competition we have data with IMDB movie reviews: the texts of the reviews and the marks (whether the review is poritive or negative). The goal is to predict the marks for reviews in test dataset.

The metric to calculate the accuracy of predictions is AUC. One characteristic of the AUC is that it is independent of the fraction of the test population which is class 0 or class 1: this makes the AUC useful for evaluating the performance of classifiers on unbalanced data sets.

In fact I simply take the texts of the reviews, drop stop words (common words, which have no impact), extract word-features and make prediction based on them.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
stops = set(stopwords.words('english')) 

In [3]:
train = pd.read_csv(r"D:\kaggle\Bag of words\labeledTrainData.tsv", header=0, delimiter='\t', quoting=3)
test = pd.read_csv(r"D:\kaggle\Bag of words\testData.tsv", header=0, delimiter='\t', quoting=3)

In [4]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [5]:
def text_to_words(text):
    """
    Extract words from text.
    """
    text = BeautifulSoup(text, 'lxml').get_text() #to remove tags. ex <br>
    letters = re.sub('[^a-zA-Z]', ' ', text)
    words = letters.lower().split()
    meaningful_words = [w for w in words if not w in stops]
    return (' '.join(meaningful_words))

In [6]:
#Check that it works
print(train['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [7]:
print(text_to_words(train['review'][0]))

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [8]:
train['review']=train.review.apply(text_to_words)
test['review']=test.review.apply(text_to_words)

In [9]:
vectorizer = CountVectorizer(analyzer = 'word',
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_df = 0.5,
                             max_features = 10000)

In [10]:
x=vectorizer.fit_transform(train.review)
x_test = vectorizer.transform(test.review)

In [11]:
xtrain, xval, ytrain, yval = train_test_split(x, train['sentiment'], test_size=0.20, random_state=36)

### Multinomial Naive Bayes model

In [12]:
mnb=MultinomialNB(alpha=0.0001)

In [13]:
y_val_m = mnb.fit(xtrain,ytrain).predict_proba(xval)[:,1]

In [14]:
roc_auc_score(yval,y_val_m)

0.9238410855634166

In [15]:
y_test_m = mnb.fit(x, train.sentiment).predict_proba(x_test)[:,1]

### Random Forest model

In [16]:
forest = RandomForestClassifier(n_estimators=300)

In [17]:
y_val_f = forest.fit(xtrain,ytrain).predict_proba(xval)[:,1]

In [18]:
roc_auc_score(yval,y_val_f)

0.9281021280571992

In [19]:
y_test_f = forest.fit(x, train.sentiment).predict_proba(x_test)[:,1]

### Ensembling of both models

In [20]:
#Ensemble of models seems to be the best.
roc_auc_score(yval, y_val_m + y_val_f)

0.9393965067163699

In [21]:
#Ensemble of models seems to be the best.
roc_auc_score(yval, (y_val_m + y_val_f)/2)

0.9393965067163699

In [22]:
y_test_e = (y_test_m+y_test_f)/2

### Output

In [23]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':y_test_e})

output.to_csv(r"D:\kaggle\Bag of words\Bag_of_Words_model.csv", index=False, quoting=3)

This competition has already ended, but people still can submit their solutions and see their scores.

My MultinomialNB model got a score of 0.90528, RandomForestClassifier 0.93064. Ensemble got a score of 0.93320.